# PROJEKTS: Knowledge Hub



Situācija
Tu esi tikko pieņemts darbā par MI Inženieri. Uzņēmuma vadītājs tev iedevis mapi UNSORTED_DATA, kurā ir haoss – sapulču ieraksti, tāfeles fotogrāfijas un līgumu melnraksti.

Nevienam nav laika to visu lasīt vai klausīties.

## Tavs Uzdevums:

Uzbūvēt Multimodālu RAG Sistēmu, kas "apēd" šos failus un ļauj direktoram čatā uzdot jautājumus (piemēram: "Cik mēs esam parādā?" vai "Ko nolēma par mārketingu?"), saņemot precīzas atbildes ar atsaucēm.

## Vides sagatavošana

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
# Instalējam nepieciešamās bibliotēkas
!pip install openai langchain langchain-community langchain-openai chromadb pypdf openai-whisper tiktoken
!apt-get install ffmpeg -y  # Nepieciešams priekš Whisper audio apstrādes

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 37 not upgraded.


In [3]:
import os
import getpass
from pathlib import Path
from google.colab import drive

# 1. Pievienojam Google Drive (ja vēl nav pievienots)
if not os.path.exists('/content/drive'):
    drive.mount('/content/drive')

# 2. Definējam darba mapi (ceļš no tava ekrānšāviņa)
BASE_PATH = Path("/content/drive/MyDrive/Colab Notebooks/BDA_M1/Lab26_KnowledgeHub")

# 3. Ievadi OpenAI API atslēgu
os.environ["OPENAI_API_KEY"] = getpass.getpass("Ievadi OpenAI API Key: ")

# 4. Pārbaudām failus
required_files = ["project.pdf", "meeting.mp3", "whiteboard.png"] # Pamanīju, ka tev ir .png
missing_files = [f for f in required_files if not (BASE_PATH / f).exists()]

if missing_files:
    print(f"❌ TRŪKST FAILU mapē {BASE_PATH}: {missing_files}")
    print("Pārbaudi failu nosaukumus!")
else:
    print(f"✅ Visi faili atrasti mapē: {BASE_PATH}")
    # Iestatām pilnos ceļus ērtākai lietošanai
    pdf_path = str(BASE_PATH / "project.pdf")
    audio_path = str(BASE_PATH / "meeting.mp3")
    image_path = str(BASE_PATH / "whiteboard.png")

✅ Visi faili atrasti mapē: /content/drive/MyDrive/Colab Notebooks/BDA_M1/Lab26_KnowledgeHub


## Izstrādes soļi (Ingestion & Vector Store)
Šis kods "apēd" failus no Google Drive mapes.
* Whisper lasa audio no Drive ceļa.
* GPT-4o lasa attēlu no Drive ceļa.
* PyPDFLoader lasa PDF no Drive ceļa.

In [4]:
import whisper
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma
from langchain_core.documents import Document
from openai import OpenAI
import base64

# Iniciējam OpenAI klientu attēlu apstrādei
client = OpenAI()

print("⏳ Sāku datu apstrādi... (Tas var aizņemt pāris minūtes)")
all_documents = []

# --- A. DATU "APĒŠANA" (Ingestion) ---

# 1. AUDIO APSTRĀDE (Whisper)
print(f"   🎤 Apstrādāju audio: {os.path.basename(audio_path)}...")
whisper_model = whisper.load_model("base")
# Whisper ņem faila ceļu tieši no Drive
result = whisper_model.transcribe(audio_path)
audio_text = result["text"]
all_documents.append(Document(page_content=audio_text, metadata={"source": "meeting.mp3"}))


# 2. ATTĒLA APSTRĀDE (GPT-4o Vision)
print(f"   🖼️ Apstrādāju attēlu: {os.path.basename(image_path)}...")

def encode_image(image_path):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode('utf-8')

# Kodējam attēlu no Drive
base64_image = encode_image(image_path)

response = client.chat.completions.create(
    model="gpt-4o",
    temperature=0,
    messages=[
        {
            "role": "user",
            "content": [
                {
                    "type": "text",
                    "text": "Extract all visible text exactly from this image. If there is a table, transcribe each row. Keep every number exactly as written (including commas, currency symbols, and dates). Then provide a short section named NUMBERS_FOUND that lists all numbers you see.",
                },
                {
                    "type": "image_url",
                    "image_url": {"url": f"data:image/png;base64,{base64_image}", "detail": "high"},
                },
            ],
        }
    ],
    max_tokens=1600,
)
image_description = response.choices[0].message.content
# Add retrieval keywords so numeric queries match this chunk more reliably.
image_text_for_index = (
    "Keywords: numbers, digits, amounts, budget, costs, estimated cost, actual, total budget, current spend.\n\n"
    f"{image_description}"
)
all_documents.append(Document(page_content=image_text_for_index, metadata={"source": "whiteboard.png"}))


# 3. PDF APSTRĀDE
print(f"   📄 Apstrādāju PDF: {os.path.basename(pdf_path)}...")
loader = PyPDFLoader(pdf_path)
pdf_docs = loader.load()
# Pievienojam PDF lapas kopējam sarakstam
all_documents.extend(pdf_docs)


# --- B. ZINĀŠANU BĀZE (Vector Store) ---

print("   💾 Veidoju Vektoru Datubāzi (ChromaDB)...")

# Sadalām tekstu gabalos
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200
)
splits = text_splitter.split_documents(all_documents)

# Saglabājam ChromaDB (operatīvajā atmiņā)
vectorstore = Chroma.from_documents(
    documents=splits,
    embedding=OpenAIEmbeddings(model="text-embedding-3-small")
)

print(f"✅ Datu bāze gatava! Apstrādāti {len(splits)} informācijas gabali.")

⏳ Sāku datu apstrādi... (Tas var aizņemt pāris minūtes)
   🎤 Apstrādāju audio: meeting.mp3...
   🖼️ Apstrādāju attēlu: whiteboard.png...


   📄 Apstrādāju PDF: project.pdf...
   💾 Veidoju Vektoru Datubāzi (ChromaDB)...
✅ Datu bāze gatava! Apstrādāti 39 informācijas gabali.


## Kods čata interfeisam (RAG funkcija)

In [5]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser


# --- RAG SISTĒMAS UZBŪVE ---

# 1. Definējam LLM
llm = ChatOpenAI(model_name="gpt-4o", temperature=0)

# 2. Definējam Meklētāju
retriever = vectorstore.as_retriever(
    search_type="mmr",
    search_kwargs={"k": 8, "fetch_k": 30, "lambda_mult": 0.15}
)

# 3. Definējam Promptu
template = """Tu esi uzņēmuma stratēģiskais AI asistents.
Tavs uzdevums ir atbildēt uz jautājumiem, izmantojot TIKAI zemāk sniegto kontekstu.
Ja atbildes nav kontekstā, saki "Diemžēl failos šādas informācijas nav."
Vienmēr norādi avota failu iekavās (piem., [Avots: meeting.mp3]).

Konteksts:
{context}

Jautājums: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

# 4. Formatēšanas funkcija (lai redzētu avotus)
def format_docs(docs):
    return "\n\n".join([f"Saturs: {d.page_content}\n[Avots: {d.metadata.get('source', 'Nezināms')}]" for d in docs])

def retrieve_docs(question):
    docs = retriever.invoke(question)
    q = question.lower()
    numeric_triggers = [
        "skait", "cipar", "budž", "izmaks", "sum", "cena", "tafel", "rēķin",
        "budget", "cost", "total", "spend", "amount", "number", "digit"
    ]
    needs_whiteboard = any(t in q for t in numeric_triggers)

    if needs_whiteboard and not any(d.metadata.get("source") == "whiteboard.png" for d in docs):
        whiteboard_docs = [d for d in splits if d.metadata.get("source") == "whiteboard.png"]
        docs = whiteboard_docs[:3] + docs

    return docs

def build_context(question):
    return format_docs(retrieve_docs(question))

# 5. RAG ķēde
rag_chain = (
    {"context": RunnableLambda(build_context), "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

# --- ČATA CILPA ---

print("🤖 Projekta Smadzenes ir gatavas. Jautā man jebko par failiem!")
print("Lai beigtu darbu, ieraksti: exit")

while True:
    # 1. Ievades lauks
    user_question = input("\nTavs Jautājums: ").strip()

    # 2. Izejas nosacījums
    if not user_question: continue
    if user_question.lower() in ["exit", "quit", "beigt", "-"]:
        print("👋 Čats pabeigts.")
        break

    # 3. RAG izpilde
    try:
        print("   🔍 Analizēju datus...")
        response = rag_chain.invoke(user_question)
        
        # 4. Rezultāts
        print(f"\nJautajums:\n{user_question}")
        print(f"\nAtbilde:\n{response}")
        print("-" * 50)
        
    except Exception as e:
        print(f"⚠️ Kļūda: {e}")

🤖 Projekta Smadzenes ir gatavas. Jautā man jebko par failiem!
Lai beigtu darbu, ieraksti: exit
   🔍 Analizēju datus...

Jautajums:
Kādi skaitļi redzami uz tāfeles/rēķina?

Atbilde:
Diemžēl failos šādas informācijas nav.
--------------------------------------------------
👋 Čats pabeigts.
